# Учебный Проект → «Прогнозирование оттока клиентов»

## Телеком — задача проекта

>Оператор связи «Ниединогоразрыва.ком» хочет научиться прогнозировать отток клиентов. Если выяснится, что пользователь планирует уйти, ему будут предложены промокоды и специальные условия. Команда оператора собрала персональные данные о некоторых клиентах, информацию об их тарифах и договорах.

✅ название и описание проекта, цель проекта

In [1]:
import pandas as pd
import re
import matplotlib as mpl
import matplotlib.pyplot as plt

Настроим вид графиков по+красоте ✨

In [2]:
%config InlineBackend.figure_formats = ['svg']

In [3]:
# style MATPLOTLIBRC
custom_params = {
                'figure.figsize': (10, 6),
                'figure.facecolor': '#232425',
                'figure.dpi': 240,

                'legend.frameon': False,
                'legend.borderpad': 1.4,
                'legend.labelspacing': 0.7,
                'legend.handlelength': 0.7,
                'legend.handleheight': 0.7,

                'axes.facecolor': '#232425',
                'axes.labelcolor': '#EEEEEE',
                'axes.labelpad': 17,
                'axes.spines.left': False,
                'axes.spines.bottom': False,
                'axes.spines.right': False,
                'axes.spines.top': False,
                'axes.grid': False,

                'contour.linewidth': 0.0,

                'xtick.color': '#AAAAAA',
                'ytick.color': '#AAAAAA',
                'xtick.bottom': True,
                'xtick.top': False,
                'ytick.left': True,
                'ytick.right': False,
    
                "lines.color": '#EEEEEE',

                'text.color': '#EEEEEE',
    
                'font.family': 'sans-serif',
            }

In [4]:
# set max columns to none
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)

Константы:

In [5]:
RANDOM_SEED = 270223
DPI_K = custom_params['figure.dpi'] / mpl.rcParams['figure.dpi']
PX = 1/custom_params['figure.dpi']
PY_CASE = re.compile(r'(?<!^)(?=[A-Z]+)')

Функции.

In [6]:
def columns_w_na(data_df):
    col_nan = data_df.columns[
            data_df.isna().any()
        ].tolist()
    df_length = len(data_df.index)

    if len(col_nan) != 0:
        print('Колонки с NaN:')
        for col in col_nan:
            count_na = len(
                    data_df.loc[data_df[col].isna()].index
                )

            print(
                '{: .2%}'.format(count_na/df_length),
                end='\t→ '
            )
            print(col, end=' → ')
            print(count_na)
    else:
        print('Пропусков NaN в наборе данных нет.\n')

In [7]:
def baisic_df_info(data_df, title='Базовая информация'):
    print(title,  '→ о наборе данных:', end='\n\n')
    print('Дубликатов:',
             len(data_df.loc[data_df.duplicated()].index),
          end='\n\n'
     )
    
    columns_w_na(data_df)
    
    data_df.info()
    
    print()
    display(
        data_df.sample(5),
        data_df.describe(),
    )

In [8]:
def to_snake_case(df, pattern=PY_CASE):
    to_return  = pd.Series(df.columns).apply(
        lambda c: re.sub(pattern, '_',  c ).lower()
    )
    
    return to_return

In [9]:
try:
    contract_df = pd.read_csv(
            './datasets/contract.csv',
        )
    internet_df = pd.read_csv(
            './datasets/internet.csv',
        )
    personal_df = pd.read_csv(
            './datasets/personal.csv',
        )
    phone_df = pd.read_csv(
            './datasets/phone.csv',
        )
except FileNotFoundError:
    contract_df = pd.read_csv(
            '/datasets/final_provider/contract.csv',
        )
    internet_df = pd.read_csv(
            '/datasets/final_provider/internet.csv',
        )
    personal_df = pd.read_csv(
            '/datasets/final_provider/personal.csv',
        )
    phone_df = pd.read_csv(
            '/datasets/final_provider/phone.csv',
        )
    print('FYI datasets loaded via url')

Сделаем названия столбцов по+красоте → `snake_case`  
Во имя Python 

In [10]:
contract_df.columns  = to_snake_case(contract_df)
internet_df.columns  = to_snake_case(internet_df)
personal_df.columns  = to_snake_case(personal_df)
phone_df.columns  = to_snake_case(phone_df)

In [11]:
baisic_df_info(contract_df, title='Контракты')

Контракты → о наборе данных:

Дубликатов: 0

Пропусков NaN в наборе данных нет.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_i_d       7043 non-null   object 
 1   begin_date         7043 non-null   object 
 2   end_date           7043 non-null   object 
 3   type               7043 non-null   object 
 4   paperless_billing  7043 non-null   object 
 5   payment_method     7043 non-null   object 
 6   monthly_charges    7043 non-null   float64
 7   total_charges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB



,customer_i_d,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges
5992,0687-ZVTHB,2014-05-01,2019-10-01 00:00:00,One year,No,Credit card (automatic),72.45,4653.85
1140,9553-DLCLU,2019-01-01,No,Two year,No,Credit card (automatic),88.95,1161.75
1625,1262-OPMFY,2019-02-01,2019-11-01 00:00:00,Month-to-month,Yes,Bank transfer (automatic),75.75,655.9
1467,8970-ANWXO,2018-03-01,No,One year,Yes,Mailed check,73.75,1756.6
4848,5380-AFSSK,2019-06-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,93.90,486.85


,monthly_charges
count,7043.000000
mean,64.761692
std,30.090047
min,18.250000
25%,35.500000
50%,70.350000
75%,89.850000
max,118.750000


In [12]:
baisic_df_info(internet_df, 'Интернет')

Интернет → о наборе данных:

Дубликатов: 0

Пропусков NaN в наборе данных нет.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   customer_i_d       5517 non-null   object
 1   internet_service   5517 non-null   object
 2   online_security    5517 non-null   object
 3   online_backup      5517 non-null   object
 4   device_protection  5517 non-null   object
 5   tech_support       5517 non-null   object
 6   streaming_t_v      5517 non-null   object
 7   streaming_movies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB



,customer_i_d,internet_service,online_security,online_backup,device_protection,tech_support,streaming_t_v,streaming_movies
4699,2887-JPYLU,DSL,No,No,No,No,No,No
4097,4971-PUYQO,Fiber optic,No,No,No,No,Yes,Yes
4906,3223-WZWJM,Fiber optic,No,No,No,No,No,Yes
2036,5837-LXSDN,DSL,No,Yes,No,No,Yes,No
1855,9755-JHNMN,DSL,Yes,Yes,Yes,Yes,No,No


,customer_i_d,internet_service,online_security,online_backup,device_protection,tech_support,streaming_t_v,streaming_movies
count,5517,5517,5517,5517,5517,5517,5517,5517
unique,5517,2,2,2,2,2,2,2
top,7590-VHVEG,Fiber optic,No,No,No,No,No,No
freq,1,3096,3498,3088,3095,3473,2810,2785


In [13]:
baisic_df_info(personal_df, title='Пользователи')

Пользователи → о наборе данных:

Дубликатов: 0

Пропусков NaN в наборе данных нет.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_i_d    7043 non-null   object
 1   gender          7043 non-null   object
 2   senior_citizen  7043 non-null   int64 
 3   partner         7043 non-null   object
 4   dependents      7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB



,customer_i_d,gender,senior_citizen,partner,dependents
1990,4514-GFCFI,Female,1,No,No
1328,6475-VHUIZ,Female,0,Yes,No
2580,7748-UMTRK,Female,1,No,Yes
6880,6976-BWGLQ,Female,0,Yes,Yes
1415,5223-UZAVK,Male,0,No,No


,senior_citizen
count,7043.000000
mean,0.162147
std,0.368612
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [14]:
baisic_df_info(phone_df, title='Телефония')

Телефония → о наборе данных:

Дубликатов: 0

Пропусков NaN в наборе данных нет.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_i_d    6361 non-null   object
 1   multiple_lines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB



,customer_i_d,multiple_lines
3490,2325-ZUSFD,No
6164,6300-BWMJX,Yes
858,6169-PPETC,No
4898,4097-YODCF,Yes
4447,8875-AKBYH,Yes


,customer_i_d,multiple_lines
count,6361,6361
unique,6361,2
top,5575-GNVDE,No
freq,1,3390


содержание (план) проекта: здесь нужно привести структуру проекта, т.е. основные пункты и подпункты. План должен быть достаточно подробным, обычном минимум 4-5 основных пунктов с подпунктами.

⭕ все датафреймы должны быть исследованы (методы describe(), info(), можно добавить графики)

⭕ вопросы и комментарии к тимлиду (если вопросов нет, нужно явно про это написать, иначе проект будет возвращен для получения возможных вопросов студента)

## План проекта

>✍️ Основные тезисы вводной консультации:
> - фиксируем параметр RANDOM_STATE = 270223 (дата начала финального спринта);
> - дедлайн сдачи отчета 11 марта 23:59 по мск (вторая суббота);
> - размер тестовой выборки: test_size = 0.25 (строго!!!);
> - рекомендуется использовать OneHotEncoder для кодирования категориальных признаков. Кодируем после разделения на train и test, чтобы не было утечки в данных;
> - если при кодировании используется get_dummies, то нужно сравнить списки получившихся признаков (должно быть полное совпадение);
> - помимо основной части, в проекте нужно сделать визуализацию графиков, исследовать корреляцию признаков. Для числовых признаков используем корреляцию Спирмена, для категориальных - Крамера. Можно применять инструмент фи-корреляции для смешанного набора признаков (библиотека phik);
> - для линейных моделей нужно масштабировать числовые признаки;
> - про баланс классов: Upsampling использовать не нужно (!!!), лучше используйте балансировку классов внутри модели;
> - на выборке test проверяется только одна лучшая модель, а сравнение моделей делаем с использованием кросс-валидации.
  
© Арсен Абдулин


- 
- 
- 

<hr>

## Основная часть проекта

<hr>

## Отчет